In [15]:
# Tensorflow and Keras should be at their latest version for good accuracy.
# Versions used here are : 
#               tensorflow Version - 2.2.0
#               keras Version - 2.4.3
# If the system is GPU enabled, CUDA and CUDnn should be installed and added to path.

In [16]:
# All necessary imports
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import Adam, RMSprop
import glob
import numpy as np

In [17]:
# Check versions
import keras
import tensorflow 
print(keras.__version__)
print(tensorflow.__version__)

2.4.3
2.2.0


In [18]:
#Here Image dimensions are in the ratio 3:2. 
#It can be changed based on object shapes.

img_width, img_height = 500, 300

In [19]:
train_data_dir = 'images/train' #train images path
val_data_dir = 'images/validation'
test_data_dir = 'images/test' #test images path
epochs = 50
batch_size = 16 

In [20]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [21]:
# Training Network (Model)

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [22]:
# Hyperparameters
sgd = RMSprop(lr=0.00001)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [23]:
# Print Summary of Model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 498, 298, 32)      896       
_________________________________________________________________
activation_5 (Activation)    (None, 498, 298, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 249, 149, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 247, 147, 64)      18496     
_________________________________________________________________
activation_6 (Activation)    (None, 247, 147, 64)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 123, 73, 64)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 121, 71, 32)      

In [24]:
# Using ImageDataGenerator to use "flow_from_directory" function which automatically..
#..labels the binary classification images.
# All identical class images should be kept in seperate folders under one train folder.
# Thus for binary classification train folder will have two sub folders.
# No need to rescale the pixel values of images

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255,)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 504 images belonging to 2 classes.
Found 86 images belonging to 2 classes.


In [25]:
#Training the model
#model.fit_generator(train_it, steps_per_epoch=16, validation_data=val_it, validation_steps=8)
model.fit( train_generator, steps_per_epoch=16, validation_data=val_generator, epochs=epochs)

Epoch 1/50
16/16 [==============================] - 71s 4s/step - loss: 0.6872 - accuracy: 0.5484 - val_loss: 0.6918 - val_accuracy: 0.5000
Epoch 2/50
16/16 [==============================] - 68s 4s/step - loss: 0.7079 - accuracy: 0.5078 - val_loss: 0.6818 - val_accuracy: 0.6047
Epoch 3/50
16/16 [==============================] - 68s 4s/step - loss: 0.6951 - accuracy: 0.5040 - val_loss: 0.6786 - val_accuracy: 0.5930
Epoch 4/50
16/16 [==============================] - 67s 4s/step - loss: 0.6881 - accuracy: 0.5391 - val_loss: 0.6792 - val_accuracy: 0.5000
Epoch 5/50
16/16 [==============================] - 67s 4s/step - loss: 0.6794 - accuracy: 0.5469 - val_loss: 0.6765 - val_accuracy: 0.5000
Epoch 6/50
16/16 [==============================] - 67s 4s/step - loss: 0.6623 - accuracy: 0.6016 - val_loss: 0.6669 - val_accuracy: 0.6628
Epoch 7/50
16/16 [==============================] - 67s 4s/step - loss: 0.6762 - accuracy: 0.5430 - val_loss: 0.6609 - val_accuracy: 0.7209
Epoch 8/50
16/16 [==

In [23]:
#Saving model
model.save('final_orientation_model.h5')

In [27]:
# Predict the output for all test images
test_names = sorted(glob.glob("images/test/test_folder/*.png"))
for name in test_names :
  img = load_img(name,target_size=(450,300))
  input_arr = img_to_array(img)
  input_arr = np.array([input_arr])  # Convert single image to a batch.
  print(int(model.predict(input_arr)))

1
0
0
0
1
1
0
1
0
1
0
1


In [26]:
# ImageDataGenerator function should not be used for predicting output for test images as it may give...
# ... wrong results. Reason for this can be found out.